In [4]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df_raw=pd.read_csv("C:\\Users\\shsubham\\program\\DATA\\titanic\\titanic_train.csv")

In [6]:
df_raw.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df=df_raw.drop(['Name','PassengerId','Ticket','Cabin','Embarked','Fare'],axis=1)

In [8]:
df.head()


,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


In [9]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [10]:
gender_encoder=OrdinalEncoder(dtype=np.int32)
gender_encoder.fit(df[['Sex']],)

OrdinalEncoder(categories='auto', dtype=<class 'numpy.int32'>)

In [11]:
df['Sex']=gender_encoder.transform(df[['Sex']])

In [12]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.0,1,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,1,0
4,0,3,1,35.0,0,0


In [13]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
dtype: int64

In [14]:
from sklearn.impute import SimpleImputer

In [15]:
SI=SimpleImputer()


In [16]:
df_final=df.dropna()

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [18]:
label=df_final['Survived']
feature=df_final.drop('Survived',axis=1)

In [19]:
X_train,X_test,y_train,y_test=train_test_split(feature,label,test_size=.3)

In [20]:
X_test.count()

Pclass    215
Sex       215
Age       215
SibSp     215
Parch     215
dtype: int64

In [21]:
LR=LogisticRegression()

In [22]:
LR.fit(X_train,y_train)

C:\Users\shsubham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
LR.coef_

array([[-0.97396851, -2.20970951, -0.02472584, -0.24145831, -0.08357872]])

In [24]:
predicted=LR.predict(X_test)

In [25]:
result_frame=pd.DataFrame({"predicted":predicted,'Actual':y_test})

In [26]:
from sklearn.metrics import f1_score

In [27]:
score=f1_score(y_test,predicted,average='weighted')

In [28]:
print(score)

0.791351596882084


In [29]:
svc= SVC(gamma='scale')

In [47]:
svc.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [48]:
svc_predict=svc.predict(X_test)

In [49]:
svc_score=f1_score(y_test,svc_predict,average='weighted')

In [50]:
print(svc_score)

0.509012044976811


In [34]:
parameter={'C':[1,2,4,8,16],'kernel':['rbf','sigmoid','poly']}

In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
gsvc=GridSearchCV(svc,parameter,cv=10)

In [37]:
gsvc.fit(X_train,y_train)

C:\Users\shsubham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 2, 4, 8, 16],
                         'kernel': ['rbf', 'sigmoid', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [38]:
gsvc.best_params_

{'C': 16, 'kernel': 'rbf'}

In [57]:
svc_best=SVC(C=1,kernel='rbf')

In [58]:
svc_best.fit(X_train,y_train)

C:\Users\shsubham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [59]:
svc_best_predict=svc_best.predict(X_test)

In [60]:
f1_svc_best=f1_score(y_test,svc_best_predict,average='weighted')

In [61]:
print(f1_svc_best)

0.7743602749106032


In [41]:
from xgboost import XGBClassifier

In [42]:
XGB=XGBClassifier()

In [43]:
XGB.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [44]:
XGB_predict=XGB.predict(X_test)

In [45]:
XGB_score=f1_score(y_test,XGB_predict,average='weighted')

In [46]:
print(XGB_score)

0.8208843831987569
